In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(100)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration_oversea import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/2.year.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['Make No'] = df_input['Make No'].astype(int)
df_input['Year'] = df_input['Year'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()

# = = = = = = = = = = = = = = =

def crawler(crawler_tuple):
    global list_dict_correct, list_series_error, count_remain

    crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

    # = = = = = = = = = = = = = = =

    try:
        count_retry = 0
        while True:
            count_retry += 1

            try:
                url_request = crawler_series['Year Url']

                resp = requests.get(url_request,
                                    headers=get_header(),
                                    proxies=get_proxy(),
                                    timeout=(5, 15))

                if resp.status_code == 200:
                    soup = BeautifulSoup(resp.text, 'lxml')
                    html = etree.HTML(str(soup))

                    if html.xpath(f'''//b/a[@href="{crawler_series['Year Url']}"]/text()''')[0] == crawler_series['Year Text']:
                        break
            except KeyboardInterrupt:
                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        list_a = html.xpath('//a[@class="subcategory_link"]')

        # = = = = = = = = = = = = = = =

        if not list_a:
            raise

        # = = = = = = = = = = = = = = =

        list_url, list_text = [a.xpath('./@href')[0].strip() for a in list_a], [a.xpath('./span[@class="subcategory_name"]/text()')[0].strip() for a in list_a]

        # = = = = = = = = = = = = = = =

        list_model = []
        for text in list_text:
            if crawler_series['Make'] in text:
                if 'TPMS' in text:
                    list_model.append(text.split(crawler_series['Make'])[1].split('TPMS')[0].strip())
                else:
                    list_model.append(text.split(crawler_series['Make'])[1].strip())
            else:
                if 'TPMS' in text:
                    list_model.append(text.split(str(crawler_series['Year']))[1].split('TPMS')[0].strip())
                else:
                    list_model.append(text.split(str(crawler_series['Year']))[1].strip())

        # = = = = = = = = = = = = = = =

        with semaphore_correct:
            list_dict_correct.extend([{'Make No': crawler_series['Make No'],
                                       'Make': crawler_series['Make'],
                                       'Year': crawler_series['Year'],
                                       'Model No': i + 1,
                                       'Model': model,
                                       'Model Text': text,
                                       'Model Url': url} for i, (model, text, url) in enumerate(zip(list_model, list_text, list_url))])

        # = = = = = = = = = = = = = = =

        crawler_status = True
    except:
        crawler_series['Request_Url'] = url_request

        with semaphore_error:
            list_series_error.append(crawler_series)

    # = = = = = = = = = = = = = = =

    if count_remain % 300 == 0 and count_remain != 0:
        clear_output()
    print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['Make No']}. {crawler_series['Make']} > {crawler_series['Year']}. {url_request}\n''')

    with semaphore_remain:
        count_remain -= 1

# = = = = = = = = = = = = = = =

pool.map(crawler, df_input.iterrows())

print('输出ing...')
print()
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Make No', 'Year', 'Model No'],
                                                             ascending=[True, False, True],
                                                             ignore_index=True)
    df_correct.to_excel(f'''./file/3.model-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['Make No', 'Year'],
                                                           ascending=[True, False],
                                                           ignore_index=True)
    df_error.to_excel(f'''./file/model_error.xlsx''', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[True] - [请求1次] - [剩余299条] - [2024/11/25 11:24:41] - 33. McLaren > 2017. https://www.tpms.com/2017-McLaren-TPMS-s/7471.htm

[True] - [请求1次] - [剩余298条] - [2024/11/25 11:24:41] - 33. McLaren > 2016. https://www.tpms.com/2016-McLaren-TPMS-s/7470.htm

[True] - [请求1次] - [剩余297条] - [2024/11/25 11:24:41] - 33. McLaren > 2015. https://www.tpms.com/2015-McLaren-TPMS-s/7469.htm

[True] - [请求1次] - [剩余296条] - [2024/11/25 11:24:41] - 32. Mazda > 2011. https://www.tpms.com/2011-Mazda-TPMS-s/2943.htm

[True] - [请求1次] - [剩余295条] - [2024/11/25 11:24:41] - 30. Maserati > 2021. https://www.tpms.com/2021-Maserati-TPMS-s/10856.htm

[True] - [请求1次] - [剩余294条] - [2024/11/25 11:24:41] - 30. Maserati > 2018. https://www.tpms.com/2018-Maserati-TPMS-s/8586.htm

[True] - [请求2次] - [剩余293条] - [2024/11/25 11:24:41] - 25. Lamborghini > 2022. https://www.tpms.com/2022-Lamborghini-TPMS-s/10895.htm

[True] - [请求1次] - [剩余292条] - [2024/11/25 11:24:41] - 34. Mercedes Benz > 2023. https://www.tpms.com/2023-Mercedes-Benz-TPM